In [1]:
import json
import pandas as pd
import glob
import datetime
from datetime import date, timedelta
import openpyxl
import pathlib
import numpy as np
from datetime import datetime
import os
import re

In [2]:
def convert_datetime(a):
    my_format="%Y-%m-%d %H:%M:%S"
    try:
        return datetime.strptime(str(a), "%Y-%m-%d %H:%M:%S").strftime(my_format)
    except:
        return datetime.strptime(str(a), "%m/%d/%Y %H:%M").strftime(my_format)

# alternation

def input_data( data_dir  ):
    data_dir = data_dir
    out_dir = '.'

    list_files = []
    for filename in pathlib.Path(data_dir).glob('**/*.csv'):
        list_files.append(filename)

    df1 = pd.concat(map(pd.read_csv, list_files),   ignore_index=True)
    
    return df1

def input_dataraw( data_dir  ):
    data_dir = data_dir
    out_dir = '.'

    list_files = []
    for filename in pathlib.Path(data_dir).glob('**/*sheet1.csv'):
        list_files.append(filename)

    df1 = pd.concat(map(pd.read_csv, list_files),   ignore_index=True)
    
    return df1

In [3]:
Rock_append = pd.concat(pd.read_excel(f"R2R3.xlsx",sheet_name=None,dtype={'Moderation Queue ID':object,
                                                                                                            'Object ID':object,
                                                                                                            'Sampling Task ID':object}),ignore_index=True)
Rock_append = Rock_append[['Sampling Queue','Moderation Queue ID','Sampling Task ID','claimant','Moderation Time',
                           'Object ID', 'Initial Moderation Result','Respondent','Moderation Time.1','Task id', 
                           'Task link','Sampling respondent Result','process result','process reason','Arbitrator',
                           'Existing tags','process reason.2','Final Arbitrator','process result.4',
                           'process result.1', 'process result.2', 'process result.3', 'process result.5','Correct Moderated Results']]

In [4]:
Rock_append = Rock_append.drop(Rock_append[Rock_append['Sampling Task ID']=='Sampling Task ID'].index)

In [5]:
#xử lý process_result
data_process_result = Rock_append[['Sampling Task ID','process result','process result.1','process result.2','process result.3','process result.4','process result.5']]

data_process_result['process result.1']=data_process_result['process result.1'].astype(str).replace('---',np.nan)
data_process_result['process result.2']=data_process_result['process result.2'].astype(str).replace('---',np.nan)
data_process_result['process result.3']=data_process_result['process result.3'].astype(str).replace('---',np.nan)
data_process_result['process result.4']=data_process_result['process result.4'].astype(str).replace('---',np.nan)
data_process_result['process result.5']=data_process_result['process result.5'].astype(str).replace('---',np.nan)

data_process_result['Sampling Task ID'] = data_process_result['Sampling Task ID'].astype('int64').astype(str)
data_process_result.set_index('Sampling Task ID',inplace=True)

def r(row):
    last_index = row.last_valid_index()
    return row[last_index] if last_index else np.nan

data_process_result['Final process result'] = data_process_result.apply(r, axis=1)
data_process_result.reset_index('Sampling Task ID',inplace=True)
data_process_result = data_process_result[['Sampling Task ID','Final process result']]
data_process_result['Final process result']=data_process_result['Final process result'].astype(str).replace('give up appealing ',"Failed appeal").astype(str)
data_process_result

C:\Users\chinh.nh1\AppData\Local\Temp\ipykernel_2424\1526918364.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_process_result['process result.1']=data_process_result['process result.1'].astype(str).replace('---',np.nan)
C:\Users\chinh.nh1\AppData\Local\Temp\ipykernel_2424\1526918364.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_process_result['process result.2']=data_process_result['process result.2'].astype(str).replace('---',np.nan)
C:\Users\chinh.nh1\AppData\Local\Temp\ipykernel_2424

,Sampling Task ID,Final process result
0,7293663745392280069,Arbitrator process overtime
1,7295864657254859270,Failed appeal


In [6]:
Rock_append[['Correct Moderated Results','Initial Moderation Result','Sampling respondent Result']] =  Rock_append[['Correct Moderated Results','Initial Moderation Result','Sampling respondent Result']].replace(to_replace=r'.+["]audit.tatus["]:["]',value= "Audit Result:: ",regex=True).astype(str)
Rock_append[['Moderated Results','Moderation Result','Respondent Result']] = Rock_append[['Correct Moderated Results','Initial Moderation Result','Sampling respondent Result']].replace(to_replace=r'^auditStatus: ',value= "Audit Result:: ",regex=True).astype(str)
Rock_append[['Moderated Results','Moderation Result','Respondent Result']] = Rock_append[['Moderated Results','Moderation Result','Respondent Result']].replace(to_replace=r'(:\s+remove)',value= ' Not Approve',regex=True).astype(str)
Rock_append[['Moderated Results','Moderation Result','Respondent Result']] = Rock_append[['Moderated Results','Moderation Result','Respondent Result']].replace(to_replace=r'(:\s+normal)',value= ' Approve',regex=True).astype(str)
Rock_append[['Moderated Results','Moderation Result','Respondent Result']] = Rock_append[['Moderated Results','Moderation Result','Respondent Result']].replace(to_replace=r'(:\s+self.isible)',value= ' Not Approve',regex=True).astype(str)

In [7]:
Rock_append['Final Result'] = Rock_append['Moderated Results'].str.extract('(Audit .esult: .+pprov[ale]{1,2})')
Rock_append['QA Result'] = Rock_append['Respondent Result'].str.extract('(Audit .esult: .+pprov[ale]{1,2})')
Rock_append['Mod Result'] = Rock_append['Moderation Result'].str.extract('(Audit .esult: .+pprov[ale]{1,2})')

Rock_append[['Final Result','QA Result','Mod Result']] = Rock_append[['Final Result','QA Result','Mod Result']].replace(to_replace='Audit .esult: ',value= '',regex=True)

In [8]:
Rock_append[['Final Result','QA Result','Mod Result']]

,Final Result,QA Result,Mod Result
0,NaN,Approval,Not approval
1,Approve,Approve,Disapprove


In [9]:
Rock_append['Final Result'] = Rock_append['Final Result'].fillna('unknown')
Rock_append['QA Result'] = Rock_append['QA Result'].fillna('unknown')
Rock_append['Mod Result'] = Rock_append['Mod Result'].fillna('unknown')


In [10]:
aliases = {"Not Approve":['"status":400','Disapproval', 'Disapproved','Not approval','Not Approval','Not Approve' ,'NotApproval','selfVisible','remove','Disapprove'] ,
    "Approve":[ '"status":200' , 'Approval', 'Approved','200' ,'200.0' ,'normal']}

for k, v in aliases.items():
    pat = '|'.join(v)
    Rock_append.loc[Rock_append['Final Result'].str.contains(pat, case=False),'Final Result'] = k
    Rock_append.loc[Rock_append['QA Result'].str.contains(pat, case=False),'QA Result'] = k
    Rock_append.loc[Rock_append['Mod Result'].str.contains(pat, case=False),'Mod Result'] = k

In [11]:
Policy_errors = [
    'Prohibited Products','Unsupported Products','Restricted Products','Adult Nudity And Sexual Activities','Hateful Behaviors','Minor Safety','Absolute Terms','Malicious Comparison','Title Spam','Abnormal Price','Inconsistent Information','Other Languages','Misleading Functionality and Effect','Redirect Traffic','Missing Brand Authorization','Unfair Trading','Obscuring Product Images','Target to Minors','Underage Commercial Behavior','False Promise','Sympathy Promotion','Misleading Size/Weight/Quantity','Misleading Patent/Endorsement/Metrics/Awards','Misleading Origin','Misleading Return & Refund Service','Misleading Guarantees and Warranties','Irrelevant Promotion','Give Away','Gambling Related Behaviors','Reproduced Content','Soliciting Mutual Interactions','Still Frame','Pirated Content','Possible Pirated Contents','Non Product Promotion Content','Vulgar Behavior','Disturbing Animal Body','Insulting and Abusive Content','Body Impurities','Dirty Environment','Eye-catching Sensitive Content','Bind A Sell B - Prohibited Product','Misleading Logistics','Promoting Counterfeit Products','Misleading Brand','Brand Circumvention','Promoting Copycat Products','Violence','Illegal Behavior','Hatespeech and Harassment','Possible IP Infringement','Spam','Invalid Company Name','Inaccurate Company Name','Inaccurate Company Address','Missing Identity Document','Expired Documentation','Black-and-White Copy','Unclear Documentation','Invalid Watermark or Logo','Incomplete Documentation','Inaccurate Representative Name','Inaccurate ID Number','Inaccurate Documentation','Inaccurate Tax Name','Inaccurate Tax Number','Inaccurate Brand Name','Inaccurate Trademark Name','Inaccurate Trademark Number','Inaccurate Application Date','Inaccurate Expiry Date','Invalid Trademark','Inaccurate Registrant','Unqualified Destination Market','Inaccurate Class(es) of Goods/Service','Unqualified Owner','Inaccurate Trademark Certificate','Expired Trademark Certificate','Unclear Trademark Certificate','Inauthentic Trademark Certificate','Unqualified 1st Level Distributor','Inaccurate 1st Level Distributor Letter','Expired 1st Level Distributor Letter','Unclear 1st Level Distributor Letter','Inauthentic 1st Level Distributor Letter','Unqualified 2nd Level Distributor','Inaccurate 2nd Level Distributor Letter','Expired 2nd Level Distributor Letter','Unclear 2nd Level Distributor Letter','Inauthentic 2nd Level Distributor Letter','Unqualified Third Level Distributor','Inaccurate Third Level Distributor Letter','Expired Third Level Distributor Letter','Unclear Third Level Distributor Letter','Inauthentic Third Level Distributor Letter','Missing Buyer Information','Missing Quantity Of Purchase Item','Missing Purchase Item Information','Missing Purchase Date','Unable To Locate Brand Item Product','Purchase Date Mismatch','Seller Information Mismatch','Missing Price Of Purchase Item','Missing Seller Information','Product Quantity Insufficient','Exceeded Validity Period','Altered Document','Lack Of Supportive Information','Unable To Recognise Source Of Purchase','Underage Applicant','Gibberish Shop Name','Abnormal Company Status','Traffic Redirection','Inaccurate Company Number','Oversea Personal Seller','PS Tampering Trace','Inauthentic Documentation','Violent Extremism','Illegal Activities','Hate Speech','Unauthorized Brand Information','Impersonation','Suicide and Dangerous Acts','Illegal Activities and Regulated Goods','Harassment and Bullying','Child Endangerment Information','Counterfeit Product Information','Inauthentic Certification of Incorporation','Inaccurate Register Date','Erotic Nudity','Illegal Activity','Hateful Contexts','Partner Name Or Logo Relates To ''TikTok''','Inauthentic Work Environment','Repeated Image','Inappropriate Watermark or Logo','Incomplete Information','Inaccurate And Misleading Review','Slightly Vulgar','Inaccurate Package Weight Information','Inappropriate Speech','Malicious Behaviors','Vulgar Issues','Imagery Depicting Skin Defects','Imagery Depicting Indecent Body Organs','Imagery Depicting Indecent Teeth&Oral','Imagery Depicting Animal Meat/Carcasses','Imagery Depicting Unpleasant Animal or Insect','Imagery Depicting Impurities','Horror Content','Scene of squalor','Offensive Content','Human Excrement','Mental Illness Content','Unsuitable for recommendation','Surprise-based Product','VAT Number Check','Slide Show','Counterfeit Word Indication','Brand Never Produced','Bind A Sell B-Prohibited Product','Pirated Content- Dedup Recall','Possible Counterfeit Product','Click-bait','Non shippable product','Physical Trademark Infringement Product','Incorrect Category-High Risk','Low cost content',
    'prohibited products','unsupported products','restricted products','adult nudity and sexual activities','hateful behaviors','minor safety','absolute terms','malicious comparison','title spam','abnormal price','inconsistent information','other languages','misleading functionality and effect','redirect traffic','missing brand authorization','unfair trading','obscuring product images','target to minors','underage commercial behavior','false promise','sympathy promotion','misleading size/weight/quantity','misleading patent/endorsement/metrics/awards','misleading origin','misleading return & refund service','misleading guarantees and warranties','irrelevant promotion','give away','gambling related behaviors','reproduced content','soliciting mutual interactions','still frame','pirated content','possible pirated contents','non product promotion content','vulgar behavior','disturbing animal body','insulting and abusive content','body impurities','dirty environment','eye-catching sensitive content','bind a sell b - prohibited product','misleading logistics','promoting counterfeit products','misleading brand','brand circumvention','promoting copycat products','violence','illegal behavior','hatespeech and harassment','possible ip infringement','spam','invalid company name','inaccurate company name','inaccurate company address','missing identity document','expired documentation','black-and-white copy','unclear documentation','invalid watermark or logo','incomplete documentation','inaccurate representative name','inaccurate id number','inaccurate documentation','inaccurate tax name','inaccurate tax number','inaccurate brand name','inaccurate trademark name','inaccurate trademark number','inaccurate application date','inaccurate expiry date','invalid trademark','inaccurate registrant','unqualified destination market','inaccurate class(es) of goods/service','unqualified owner','inaccurate trademark certificate','expired trademark certificate','unclear trademark certificate','inauthentic trademark certificate','unqualified 1st level distributor','inaccurate 1st level distributor letter','expired 1st level distributor letter','unclear 1st level distributor letter','inauthentic 1st level distributor letter','unqualified 2nd level distributor','inaccurate 2nd level distributor letter','expired 2nd level distributor letter','unclear 2nd level distributor letter','inauthentic 2nd level distributor letter','unqualified third level distributor','inaccurate third level distributor letter','expired third level distributor letter','unclear third level distributor letter','inauthentic third level distributor letter','missing buyer information','missing quantity of purchase item','missing purchase item information','missing purchase date','unable to locate brand item product','purchase date mismatch','seller information mismatch','missing price of purchase item','missing seller information','product quantity insufficient','exceeded validity period','altered document','lack of supportive information','unable to recognise source of purchase','underage applicant','gibberish shop name','abnormal company status','traffic redirection','inaccurate company number','oversea personal seller','ps tampering trace','inauthentic documentation','violent extremism','illegal activities','hate speech','unauthorized brand information','impersonation','suicide and dangerous acts','illegal activities and regulated goods','harassment and bullying','child endangerment information','counterfeit product information','inauthentic certification of incorporation','inaccurate register date','erotic nudity','illegal activity','hateful contexts','partner name or logo relates to ''tiktok''','inauthentic work environment','repeated image','inappropriate watermark or logo','incomplete information','inaccurate and misleading review','slightly vulgar','inaccurate package weight information','inappropriate speech','malicious behaviors','vulgar issues','imagery depicting skin defects','imagery depicting indecent body organs','imagery depicting indecent teeth&oral','imagery depicting animal meat/carcasses','imagery depicting unpleasant animal or insect','imagery depicting impurities','horror content','scene of squalor','offensive content','human excrement','mental illness content','unsuitable for recommendation','surprise-based product','vat number check','slide show','counterfeit word indication','brand never produced','bind a sell b-prohibited product','pirated content- dedup recall','possible counterfeit product','click-bait','non shippable product','physical trademark infringement product','incorrect category-high risk','low cost content',
    'Prohibited products','Unsupported products','Restricted products','Adult nudity and sexual activities','Hateful behaviors','Minor safety','Absolute terms','Malicious comparison','Title spam','Abnormal price','Inconsistent information','Other languages','Misleading functionality and effect','Redirect traffic','Missing brand authorization','Unfair trading','Obscuring product images','Target to minors','Underage commercial behavior','False promise','Sympathy promotion','Misleading size/weight/quantity','Misleading patent/endorsement/metrics/awards','Misleading origin','Misleading return & refund service','Misleading guarantees and warranties','Irrelevant promotion','Give away','Gambling related behaviors','Reproduced content','Soliciting mutual interactions','Still frame','Pirated content','Possible pirated contents','Non product promotion content','Vulgar behavior','Disturbing animal body','Insulting and abusive content','Body impurities','Dirty environment','Eye-catching sensitive content','Bind a sell b - prohibited product','Misleading logistics','Promoting counterfeit products','Misleading brand','Brand circumvention','Promoting copycat products','Violence','Illegal behavior','Hatespeech and harassment','Possible ip infringement','Spam','Invalid company name','Inaccurate company name','Inaccurate company address','Missing identity document','Expired documentation','Black-and-white copy','Unclear documentation','Invalid watermark or logo','Incomplete documentation','Inaccurate representative name','Inaccurate id number','Inaccurate documentation','Inaccurate tax name','Inaccurate tax number','Inaccurate brand name','Inaccurate trademark name','Inaccurate trademark number','Inaccurate application date','Inaccurate expiry date','Invalid trademark','Inaccurate registrant','Unqualified destination market','Inaccurate class(es) of goods/service','Unqualified owner','Inaccurate trademark certificate','Expired trademark certificate','Unclear trademark certificate','Inauthentic trademark certificate','Unqualified 1st level distributor','Inaccurate 1st level distributor letter','Expired 1st level distributor letter','Unclear 1st level distributor letter','Inauthentic 1st level distributor letter','Unqualified 2nd level distributor','Inaccurate 2nd level distributor letter','Expired 2nd level distributor letter','Unclear 2nd level distributor letter','Inauthentic 2nd level distributor letter','Unqualified third level distributor','Inaccurate third level distributor letter','Expired third level distributor letter','Unclear third level distributor letter','Inauthentic third level distributor letter','Missing buyer information','Missing quantity of purchase item','Missing purchase item information','Missing purchase date','Unable to locate brand item product','Purchase date mismatch','Seller information mismatch','Missing price of purchase item','Missing seller information','Product quantity insufficient','Exceeded validity period','Altered document','Lack of supportive information','Unable to recognise source of purchase','Underage applicant','Gibberish shop name','Abnormal company status','Traffic redirection','Inaccurate company number','Oversea personal seller','Ps tampering trace','Inauthentic documentation','Violent extremism','Illegal activities','Hate speech','Unauthorized brand information','Impersonation','Suicide and dangerous acts','Illegal activities and regulated goods','Harassment and bullying','Child endangerment information','Counterfeit product information','Inauthentic certification of incorporation','Inaccurate register date','Erotic nudity','Illegal activity','Hateful contexts','Partner name or logo relates to ''tiktok''','Inauthentic work environment','Repeated image','Inappropriate watermark or logo','Incomplete information','Inaccurate and misleading review','Slightly vulgar','Inaccurate package weight information','Inappropriate speech','Malicious behaviors','Vulgar issues','Imagery depicting skin defects','Imagery depicting indecent body organs','Imagery depicting indecent teeth&oral','Imagery depicting animal meat/carcasses','Imagery depicting unpleasant animal or insect','Imagery depicting impurities','Horror content','Scene of squalor','Offensive content','Human excrement','Mental illness content','Unsuitable for recommendation','Surprise-based product','Vat number check','Slide show','Counterfeit word indication','Brand never produced','Bind a sell b-prohibited product','Pirated content- dedup recall','Possible counterfeit product','Click-bait','Non shippable product','Physical trademark infringement product','Incorrect category-high risk','Low cost content',
                
    'Gambling related behaviour','gambling related behaviour','Gambling Related Behaviour',
    'copycat products','Copycat products','Copycat Products',
    'normal','remove','selfVisible'
    ]

In [12]:
Rock_append['Correct Moderated Results'] = Rock_append['Correct Moderated Results'].str.lower()
Rock_append['Sampling respondent Result'] = Rock_append['Sampling respondent Result'].str.lower()
Rock_append['Initial Moderation Result'] = Rock_append['Initial Moderation Result'].str.lower()

In [13]:
Policy_errors_pattern = "|".join(Policy_errors )

def pattern_searcher(search_str:str, search_list:str):

    search_obj = re.search(search_list, search_str)
    if search_obj :
        return_str = search_str[search_obj.start(): search_obj.end()]
    else:
        return_str = np.nan
    return return_str

In [14]:
Rock_append['Policy Errors'] = Rock_append['Correct Moderated Results'].astype(str).apply(lambda x: pattern_searcher(search_str=x, search_list=Policy_errors_pattern))
Rock_append['QA Reason'] = Rock_append['Sampling respondent Result'].astype(str).apply(lambda y: pattern_searcher(search_str=y, search_list=Policy_errors_pattern))
Rock_append['Mod Reason'] = Rock_append['Initial Moderation Result'].astype(str).apply(lambda z: pattern_searcher(search_str=z, search_list=Policy_errors_pattern))

In [15]:
Rock_append['Policy Errors']

0   NaN
1   NaN
Name: Policy Errors, dtype: float64

In [16]:
def d(row):
    last_index = row.last_valid_index()
    return row[last_index] if last_index else np.nan


In [17]:
#Final Policy Errors
Rock_append['Final policy errors'] = Rock_append[['Mod Reason','QA Reason','Policy Errors']].apply(d, axis=1)
Rock_append['Final policy errors']

0    lack of supportive information
1             unclear documentation
Name: Final policy errors, dtype: object

In [18]:
# Splitting the RCA labels
try:
    Rock_append[['RCA lvl1','RCA lvl2','RCA lvl3']] = Rock_append['Existing tags'].str.split(' > ', expand=True, n=2)
except:
    Rock_append[['RCA lvl1','RCA lvl2','RCA lvl3']] = np.nan

In [19]:
#Add diff case
Rock_append.loc[Rock_append['Mod Result'] != Rock_append['QA Result'],'Diff case']="FALSE"
Rock_append.loc[Rock_append['Diff case'].isnull(),'Diff case']="TRUE"

C:\Users\chinh.nh1\AppData\Local\Temp\ipykernel_2424\3741926143.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'FALSE' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  Rock_append.loc[Rock_append['Mod Result'] != Rock_append['QA Result'],'Diff case']="FALSE"


In [20]:
#FN/FP
Rock_append.loc[(Rock_append['QA Result'] == 'Approve') & (Rock_append['process result'] == 'Failed appeal'),'Final False Error']="False Positive"
Rock_append.loc[Rock_append['Final False Error'].isnull(),'Final False Error']="False Negative"
Rock_append.loc[(Rock_append['Mod Result'] == 'Approve') & (Rock_append['process result'] == 'Appeal Successsfully'),'Final False Error']="False Positive"
Rock_append.loc[Rock_append['Final False Error'].isnull(),'Final False Error']="False Negative"
Rock_append.loc[(Rock_append['QA Result'] == 'Approve') & (Rock_append['process result'] == 'Edge case'),'Final False Error']="False Positive"
Rock_append.loc[Rock_append['Final False Error'].isnull(),'Final False Error']="False Negative"
Rock_append.loc[Rock_append['process result'] == '---','Final False Error']="---"

C:\Users\chinh.nh1\AppData\Local\Temp\ipykernel_2424\4026043698.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False Positive' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  Rock_append.loc[(Rock_append['QA Result'] == 'Approve') & (Rock_append['process result'] == 'Failed appeal'),'Final False Error']="False Positive"


In [21]:
#Convert_time
Rock_append['Moderation Time'] = Rock_append['Moderation Time'].apply(convert_datetime)
Rock_append['Moderation Time'] = pd.to_datetime(Rock_append['Moderation Time'])

Rock_append['Moderation Time.1'] = Rock_append['Moderation Time.1'].apply(convert_datetime)
Rock_append['Moderation Time.1'] = pd.to_datetime(Rock_append['Moderation Time.1'])

In [22]:
Final_Rock_append = Rock_append[['Sampling Queue','Moderation Queue ID','claimant','Moderation Time','Object ID','Mod Result', 
                   'Mod Reason','Respondent','Moderation Time.1','Sampling Task ID','Task link','QA Result', 'QA Reason','Diff case','process result',
                   'process reason','Arbitrator','Final False Error','Final policy errors','RCA lvl1','RCA lvl2','RCA lvl3','process reason.2',
                   'Final Arbitrator','process result.4','Final Result']]

Final_Rock_append['Sampling Task ID'] = Final_Rock_append['Sampling Task ID'].astype('int64').astype(str)
Final_Rock_append.columns

C:\Users\chinh.nh1\AppData\Local\Temp\ipykernel_2424\3126434807.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Final_Rock_append['Sampling Task ID'] = Final_Rock_append['Sampling Task ID'].astype('int64').astype(str)


Index(['Sampling Queue', 'Moderation Queue ID', 'claimant', 'Moderation Time',
       'Object ID', 'Mod Result', 'Mod Reason', 'Respondent',
       'Moderation Time.1', 'Sampling Task ID', 'Task link', 'QA Result',
       'QA Reason', 'Diff case', 'process result', 'process reason',
       'Arbitrator', 'Final False Error', 'Final policy errors', 'RCA lvl1',
       'RCA lvl2', 'RCA lvl3', 'process reason.2', 'Final Arbitrator',
       'process result.4', 'Final Result'],
      dtype='object')

In [23]:
Final_Rock_append = pd.merge(Final_Rock_append,data_process_result,how='inner',on=['Sampling Task ID'])

Final_Rock_append['Object ID'] = "id=" + Final_Rock_append['Object ID'].astype(str)
Final_Rock_append['Sampling Task ID'] = "id=" + Final_Rock_append['Sampling Task ID'].astype('int64').astype(str)
Final_Rock_append['Moderation Queue ID'] = "id=" + Final_Rock_append['Moderation Queue ID'].astype('int64').astype(str)
Final_Rock_append.drop_duplicates(subset='Sampling Task ID',keep='last')
Final_Rock_append

,Sampling Queue,Moderation Queue ID,claimant,Moderation Time,Object ID,Mod Result,Mod Reason,Respondent,Moderation Time.1,Sampling Task ID,...,Final False Error,Final policy errors,RCA lvl1,RCA lvl2,RCA lvl3,process reason.2,Final Arbitrator,process result.4,Final Result,Final process result
0,Inhouse QA VN LL Trademark New,id=7173977496721932802,yen.th@trans-cosmos.com.vn,2023-10-24 20:24:10,id=7292987330841921285,Not Approve,lack of supportive information,***,2023-10-25 11:25:57,id=7293663745392280069,...,False Negative,lack of supportive information,2. SOP,2.1 Policy Ambiguity,2.1.4 Policy update timeline gap,无人认领超时/Unclaimed timeout,NaN,---,unknown,Arbitrator process overtime
1,Inhouse QA VN LL Seller On Boarding,id=7091496333818839554,duy.nt2@trans-cosmos.com.vn,2023-10-30 20:53:50,id=7495365688557144227,Not Approve,unclear documentation,***,2023-10-31 09:48:11,id=7295864657254859270,...,False Negative,unclear documentation,1. People,1.4 Content Misinterpretation,1.4.1 Misinterpretation of the content,"hình ảnh ID rõ ràng, không bị unclear nhé",NaN,---,Approve,Failed appeal


In [24]:
Final_Rock_append.columns

Index(['Sampling Queue', 'Moderation Queue ID', 'claimant', 'Moderation Time',
       'Object ID', 'Mod Result', 'Mod Reason', 'Respondent',
       'Moderation Time.1', 'Sampling Task ID', 'Task link', 'QA Result',
       'QA Reason', 'Diff case', 'process result', 'process reason',
       'Arbitrator', 'Final False Error', 'Final policy errors', 'RCA lvl1',
       'RCA lvl2', 'RCA lvl3', 'process reason.2', 'Final Arbitrator',
       'process result.4', 'Final Result', 'Final process result'],
      dtype='object')

In [25]:
Final_Rock_append = Final_Rock_append[['Sampling Queue','Sampling Task ID','claimant','Moderation Time','Mod Result', 
                   'Mod Reason','Object ID','Respondent','Moderation Time.1','Task link', 'QA Result', 'QA Reason','Diff case','process result',
                   'process reason','Arbitrator','Final Result','Final process result','Final False Error','Final policy errors','RCA lvl1','RCA lvl2','RCA lvl3','process reason.2',
                   'Final Arbitrator','process result.4']]

In [26]:
try:
    Final_Rock_append.to_excel('R2R3 Merged.xlsx')
except:
    print('Error')